# W_GL_ACCOUNT_D ETL Process
### Staging to Dimension - Source-Dated SCD Type 2 with NOT_EXISTS Detection
**ODI Package:** SILOS_SIL_GLACCOUNTDIMENSION  
**IKM:** BIAPPS Oracle Slowly Changing Dimension  
**Detection Strategy:** NOT_EXISTS (insert only new versions based on SRC_EFF_FROM_DT + natural key)

In [ ]:
%sql
-- Step 1: Create Flow Table I$_2625278_1 with all transformations
-- This table contains only NEW records that don't exist in target (NOT_EXISTS logic)
DROP VIEW IF EXISTS I$_2625278_1;

CREATE OR REPLACE TEMP VIEW I$_2625278_1 AS
SELECT
  -- Domain mapping for GL_ACCOUNT_CAT_CODE
  CASE
    WHEN COALESCE(S.GL_ACCOUNT_CAT_CODE, '__NOT_APPLICABLE__') = '__NOT_APPLICABLE__' THEN
      CASE
        WHEN COALESCE(
          (SELECT TRG_DOMAIN_MEMBER_CODE FROM workspace.oracle_edw.W_DOMAIN_MEMBER_MAP_G
           WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
             AND SRC_DOMAIN_MEMBER_CODE = COALESCE(COALESCE(S.GROUP_ACCOUNT_NUM, LKP_NAT_ACCT.GROUP_ACCOUNT_NUM), '__UNASSIGNED__')
             AND SRC_DATASOURCE_NUM_ID IN (999, 999)
             AND TRG_DOMAIN_CODE = 'W_GL_ACCT_CATEGORY'),
          (SELECT TRG_DOMAIN_MEMBER_CODE FROM workspace.oracle_edw.W_DOMAIN_MEMBER_MAP_G
           WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
             AND SRC_DOMAIN_MEMBER_CODE = '__ANY__'
             AND SRC_DATASOURCE_NUM_ID IN (999, 999)
             AND TRG_DOMAIN_CODE = 'W_GL_ACCT_CATEGORY'),
          '__ERROR__'
        ) = '__ERROR__' THEN 'BS'
        ELSE COALESCE(
          (SELECT TRG_DOMAIN_MEMBER_CODE FROM workspace.oracle_edw.W_DOMAIN_MEMBER_MAP_G
           WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
             AND SRC_DOMAIN_MEMBER_CODE = COALESCE(COALESCE(S.GROUP_ACCOUNT_NUM, LKP_NAT_ACCT.GROUP_ACCOUNT_NUM), '__UNASSIGNED__')
             AND SRC_DATASOURCE_NUM_ID IN (999, 999)
             AND TRG_DOMAIN_CODE = 'W_GL_ACCT_CATEGORY'),
          'BS'
        )
      END
    ELSE COALESCE(S.GL_ACCOUNT_CAT_CODE, '__NOT_APPLICABLE__')
  END AS GL_ACCOUNT_CAT_CODE,
  
  -- Domain mapping for FIN_STMT_ITEM_CODE
  CASE
    WHEN COALESCE(S.FIN_STMT_ITEM_CODE, '__NOT_APPLICABLE__') = '__NOT_APPLICABLE__' THEN
      CASE
        WHEN COALESCE(
          (SELECT TRG_DOMAIN_MEMBER_CODE FROM workspace.oracle_edw.W_DOMAIN_MEMBER_MAP_G
           WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
             AND SRC_DOMAIN_MEMBER_CODE = COALESCE(COALESCE(S.GROUP_ACCOUNT_NUM, LKP_NAT_ACCT.GROUP_ACCOUNT_NUM), '__UNASSIGNED__')
             AND SRC_DATASOURCE_NUM_ID IN (999, 999)
             AND TRG_DOMAIN_CODE = 'W_FIN_STMT'),
          (SELECT TRG_DOMAIN_MEMBER_CODE FROM workspace.oracle_edw.W_DOMAIN_MEMBER_MAP_G
           WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
             AND SRC_DOMAIN_MEMBER_CODE = '__ANY__'
             AND SRC_DATASOURCE_NUM_ID IN (999, 999)
             AND TRG_DOMAIN_CODE = 'W_FIN_STMT'),
          '__ERROR__'
        ) = '__ERROR__' THEN 'OTHERS'
        ELSE COALESCE(
          (SELECT TRG_DOMAIN_MEMBER_CODE FROM workspace.oracle_edw.W_DOMAIN_MEMBER_MAP_G
           WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
             AND SRC_DOMAIN_MEMBER_CODE = COALESCE(COALESCE(S.GROUP_ACCOUNT_NUM, LKP_NAT_ACCT.GROUP_ACCOUNT_NUM), '__UNASSIGNED__')
             AND SRC_DATASOURCE_NUM_ID IN (999, 999)
             AND TRG_DOMAIN_CODE = 'W_FIN_STMT'),
          'OTHERS'
        )
      END
    ELSE COALESCE(S.FIN_STMT_ITEM_CODE, '__NOT_APPLICABLE__')
  END AS FIN_STMT_ITEM_CODE,
  
  COALESCE(S.CHART_OF_ACCOUNTS, '__NOT_APPLICABLE__') AS CHART_OF_ACCOUNTS,
  S.CONCATENATED_SEGMENT_VALUE,
  S.PROFIT_CENTER_NUM,
  S.PROFIT_CENTER_ATTRIB,
  S.BALANCING_SEGMENT_NUM,
  S.BALANCING_SEGMENT_ATTRIB,
  S.COST_CENTER_NUM,
  S.COST_CENTER_ATTRIB,
  S.NATURAL_ACCOUNT_NUM,
  S.NATURAL_ACCOUNT_ATTRIB,
  S.MANAGEMENT_SEGMENT_NUM,
  COALESCE(S.COMPANY_CODE, '__NOT_APPLICABLE__') AS COMPANY_CODE,
  S.COMPANY_NAME,
  COALESCE(COALESCE(S.GROUP_ACCOUNT_NUM, LKP_NAT_ACCT.GROUP_ACCOUNT_NUM), 'OTHERS') AS GROUP_ACCOUNT_NUM,
  COALESCE(S.RECON_TYPE_CODE, '__NOT_APPLICABLE__') AS RECON_TYPE_CODE,
  COALESCE(S.TAX_CAT_CODE, '__NOT_APPLICABLE__') AS TAX_CAT_CODE,
  S.TAX_CAT_NAME,
  COALESCE(S.CURRENCY_CODE, '__NOT_APPLICABLE__') AS CURRENCY_CODE,
  S.CURRENCY_NAME,
  S.ACTIVE_FLG,
  
  -- All 30 account segments
  COALESCE(S.ACCOUNT_SEG1_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG1_CODE,
  S.ACCOUNT_SEG1_ATTRIB,
  CONCAT(S.ACCOUNT_SEG1_ATTRIB, '~', S.ACCOUNT_SEG1_CODE) AS ACCOUNT_SEG1_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG2_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG2_CODE,
  S.ACCOUNT_SEG2_ATTRIB,
  CONCAT(S.ACCOUNT_SEG2_ATTRIB, '~', S.ACCOUNT_SEG2_CODE) AS ACCOUNT_SEG2_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG3_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG3_CODE,
  S.ACCOUNT_SEG3_ATTRIB,
  CONCAT(S.ACCOUNT_SEG3_ATTRIB, '~', S.ACCOUNT_SEG3_CODE) AS ACCOUNT_SEG3_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG4_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG4_CODE,
  S.ACCOUNT_SEG4_ATTRIB,
  CONCAT(S.ACCOUNT_SEG4_ATTRIB, '~', S.ACCOUNT_SEG4_CODE) AS ACCOUNT_SEG4_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG5_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG5_CODE,
  S.ACCOUNT_SEG5_ATTRIB,
  CONCAT(S.ACCOUNT_SEG5_ATTRIB, '~', S.ACCOUNT_SEG5_CODE) AS ACCOUNT_SEG5_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG6_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG6_CODE,
  S.ACCOUNT_SEG6_ATTRIB,
  CONCAT(S.ACCOUNT_SEG6_ATTRIB, '~', S.ACCOUNT_SEG6_CODE) AS ACCOUNT_SEG6_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG7_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG7_CODE,
  S.ACCOUNT_SEG7_ATTRIB,
  CONCAT(S.ACCOUNT_SEG7_ATTRIB, '~', S.ACCOUNT_SEG7_CODE) AS ACCOUNT_SEG7_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG8_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG8_CODE,
  S.ACCOUNT_SEG8_ATTRIB,
  CONCAT(S.ACCOUNT_SEG8_ATTRIB, '~', S.ACCOUNT_SEG8_CODE) AS ACCOUNT_SEG8_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG9_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG9_CODE,
  S.ACCOUNT_SEG9_ATTRIB,
  CONCAT(S.ACCOUNT_SEG9_ATTRIB, '~', S.ACCOUNT_SEG9_CODE) AS ACCOUNT_SEG9_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG10_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG10_CODE,
  S.ACCOUNT_SEG10_ATTRIB,
  CONCAT(S.ACCOUNT_SEG10_ATTRIB, '~', S.ACCOUNT_SEG10_CODE) AS ACCOUNT_SEG10_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG11_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG11_CODE,
  S.ACCOUNT_SEG11_ATTRIB,
  CONCAT(S.ACCOUNT_SEG11_ATTRIB, '~', S.ACCOUNT_SEG11_CODE) AS ACCOUNT_SEG11_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG12_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG12_CODE,
  S.ACCOUNT_SEG12_ATTRIB,
  CONCAT(S.ACCOUNT_SEG12_ATTRIB, '~', S.ACCOUNT_SEG12_CODE) AS ACCOUNT_SEG12_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG13_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG13_CODE,
  S.ACCOUNT_SEG13_ATTRIB,
  CONCAT(S.ACCOUNT_SEG13_ATTRIB, '~', S.ACCOUNT_SEG13_CODE) AS ACCOUNT_SEG13_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG14_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG14_CODE,
  S.ACCOUNT_SEG14_ATTRIB,
  CONCAT(S.ACCOUNT_SEG14_ATTRIB, '~', S.ACCOUNT_SEG14_CODE) AS ACCOUNT_SEG14_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG15_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG15_CODE,
  S.ACCOUNT_SEG15_ATTRIB,
  CONCAT(S.ACCOUNT_SEG15_ATTRIB, '~', S.ACCOUNT_SEG15_CODE) AS ACCOUNT_SEG15_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG16_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG16_CODE,
  S.ACCOUNT_SEG16_ATTRIB,
  CONCAT(S.ACCOUNT_SEG16_ATTRIB, '~', S.ACCOUNT_SEG16_CODE) AS ACCOUNT_SEG16_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG17_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG17_CODE,
  S.ACCOUNT_SEG17_ATTRIB,
  CONCAT(S.ACCOUNT_SEG17_ATTRIB, '~', S.ACCOUNT_SEG17_CODE) AS ACCOUNT_SEG17_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG18_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG18_CODE,
  S.ACCOUNT_SEG18_ATTRIB,
  CONCAT(S.ACCOUNT_SEG18_ATTRIB, '~', S.ACCOUNT_SEG18_CODE) AS ACCOUNT_SEG18_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG19_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG19_CODE,
  S.ACCOUNT_SEG19_ATTRIB,
  CONCAT(S.ACCOUNT_SEG19_ATTRIB, '~', S.ACCOUNT_SEG19_CODE) AS ACCOUNT_SEG19_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG20_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG20_CODE,
  S.ACCOUNT_SEG20_ATTRIB,
  CONCAT(S.ACCOUNT_SEG20_ATTRIB, '~', S.ACCOUNT_SEG20_CODE) AS ACCOUNT_SEG20_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG21_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG21_CODE,
  S.ACCOUNT_SEG21_ATTRIB,
  CONCAT(S.ACCOUNT_SEG21_ATTRIB, '~', S.ACCOUNT_SEG21_CODE) AS ACCOUNT_SEG21_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG22_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG22_CODE,
  S.ACCOUNT_SEG22_ATTRIB,
  CONCAT(S.ACCOUNT_SEG22_ATTRIB, '~', S.ACCOUNT_SEG22_CODE) AS ACCOUNT_SEG22_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG23_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG23_CODE,
  S.ACCOUNT_SEG23_ATTRIB,
  CONCAT(S.ACCOUNT_SEG23_ATTRIB, '~', S.ACCOUNT_SEG23_CODE) AS ACCOUNT_SEG23_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG24_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG24_CODE,
  S.ACCOUNT_SEG24_ATTRIB,
  CONCAT(S.ACCOUNT_SEG24_ATTRIB, '~', S.ACCOUNT_SEG24_CODE) AS ACCOUNT_SEG24_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG25_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG25_CODE,
  S.ACCOUNT_SEG25_ATTRIB,
  CONCAT(S.ACCOUNT_SEG25_ATTRIB, '~', S.ACCOUNT_SEG25_CODE) AS ACCOUNT_SEG25_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG26_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG26_CODE,
  S.ACCOUNT_SEG26_ATTRIB,
  CONCAT(S.ACCOUNT_SEG26_ATTRIB, '~', S.ACCOUNT_SEG26_CODE) AS ACCOUNT_SEG26_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG27_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG27_CODE,
  S.ACCOUNT_SEG27_ATTRIB,
  CONCAT(S.ACCOUNT_SEG27_ATTRIB, '~', S.ACCOUNT_SEG27_CODE) AS ACCOUNT_SEG27_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG28_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG28_CODE,
  S.ACCOUNT_SEG28_ATTRIB,
  CONCAT(S.ACCOUNT_SEG28_ATTRIB, '~', S.ACCOUNT_SEG28_CODE) AS ACCOUNT_SEG28_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG29_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG29_CODE,
  S.ACCOUNT_SEG29_ATTRIB,
  CONCAT(S.ACCOUNT_SEG29_ATTRIB, '~', S.ACCOUNT_SEG29_CODE) AS ACCOUNT_SEG29_INTEGRATION_ID,
  COALESCE(S.ACCOUNT_SEG30_CODE, '__NOT_APPLICABLE__') AS ACCOUNT_SEG30_CODE,
  S.ACCOUNT_SEG30_ATTRIB,
  CONCAT(S.ACCOUNT_SEG30_ATTRIB, '~', S.ACCOUNT_SEG30_CODE) AS ACCOUNT_SEG30_INTEGRATION_ID,
  
  COALESCE(LKP_USER_CREATED.ROW_WID, 0) AS CREATED_BY_WID,
  COALESCE(LKP_USER_CHANGED.ROW_WID, 0) AS CHANGED_BY_WID,
  S.CREATED_ON_DT,
  S.CHANGED_ON_DT,
  S.AUX1_CHANGED_ON_DT,
  S.AUX2_CHANGED_ON_DT,
  S.AUX3_CHANGED_ON_DT,
  S.AUX4_CHANGED_ON_DT,
  S.SRC_EFF_FROM_DT,
  S.SRC_EFF_TO_DT,
  CASE WHEN S.DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
  S.DATASOURCE_NUM_ID,
  S.INTEGRATION_ID,
  S.SET_ID,
  S.TENANT_ID,
  S.X_CUSTOM,
  COALESCE(S.PROJECT_CODE, '__NOT_APPLICABLE__') AS PROJECT_CODE,
  S.PROJECT_ATTRIB,
  S.PROJECT_INTEGRATION_ID,
  COALESCE(S.PROGRAM_CODE, '__NOT_APPLICABLE__') AS PROGRAM_CODE,
  S.PROGRAM_ATTRIB,
  S.PROGRAM_INTEGRATION_ID,
  
  -- SCD Type 2 columns
  COALESCE(S.SRC_EFF_FROM_DT, CAST('1700-01-01' AS TIMESTAMP)) AS EFFECTIVE_FROM_DT,
  CAST('9999-12-31' AS TIMESTAMP) AS EFFECTIVE_TO_DT,
  'Y' AS CURRENT_FLG,
  'I' AS IND_UPDATE
  
FROM workspace.oracle_edw.W_GL_ACCOUNT_DS S

-- Lookup to W_NATURAL_ACCOUNT_D for GROUP_ACCOUNT_NUM
LEFT OUTER JOIN workspace.oracle_edw.W_NATURAL_ACCOUNT_D LKP_NAT_ACCT
  ON S.DATASOURCE_NUM_ID = LKP_NAT_ACCT.DATASOURCE_NUM_ID
  AND S.NATURAL_ACCOUNT_NUM = LKP_NAT_ACCT.NATURAL_ACCOUNT_VAL_CODE
  AND S.NATURAL_ACCOUNT_ATTRIB = LKP_NAT_ACCT.NATURAL_ACCOUNT_LOV_ID
  AND S.SRC_EFF_FROM_DT >= LKP_NAT_ACCT.EFFECTIVE_FROM_DT
  AND S.SRC_EFF_FROM_DT < LKP_NAT_ACCT.EFFECTIVE_TO_DT

-- User lookups for CREATED_BY and CHANGED_BY
LEFT OUTER JOIN workspace.oracle_edw.W_USER_D LKP_USER_CREATED
  ON S.DATASOURCE_NUM_ID = LKP_USER_CREATED.DATASOURCE_NUM_ID
  AND S.CREATED_BY_ID = LKP_USER_CREATED.INTEGRATION_ID
  AND S.CREATED_ON_DT >= LKP_USER_CREATED.EFFECTIVE_FROM_DT
  AND S.CREATED_ON_DT < LKP_USER_CREATED.EFFECTIVE_TO_DT
  AND LKP_USER_CREATED.DELETE_FLG = 'N'
  
LEFT OUTER JOIN workspace.oracle_edw.W_USER_D LKP_USER_CHANGED
  ON S.DATASOURCE_NUM_ID = LKP_USER_CHANGED.DATASOURCE_NUM_ID
  AND S.CHANGED_BY_ID = LKP_USER_CHANGED.INTEGRATION_ID
  AND S.CHANGED_ON_DT >= LKP_USER_CHANGED.EFFECTIVE_FROM_DT
  AND S.CHANGED_ON_DT < LKP_USER_CHANGED.EFFECTIVE_TO_DT
  AND LKP_USER_CHANGED.DELETE_FLG = 'N'

-- NOT EXISTS filter: Only insert records that don't exist in target
WHERE NOT EXISTS (
  SELECT 1 FROM workspace.oracle_edw.W_GL_ACCOUNT_D T
  WHERE T.SRC_EFF_FROM_DT = S.SRC_EFF_FROM_DT
    AND T.DATASOURCE_NUM_ID = S.DATASOURCE_NUM_ID
    AND T.INTEGRATION_ID = S.INTEGRATION_ID
    -- Change detection on all timestamp columns
    AND COALESCE(T.CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(S.CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
    AND COALESCE(T.AUX1_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(S.AUX1_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
    AND COALESCE(T.AUX2_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(S.AUX2_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
    AND COALESCE(T.AUX3_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(S.AUX3_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
    AND COALESCE(T.AUX4_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(S.AUX4_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
)

In [ ]:
%sql
-- Step 2: Insert new rows into W_GL_ACCOUNT_D
-- Only INSERT, no UPDATE (NOT_EXISTS strategy)
INSERT INTO workspace.oracle_edw.W_GL_ACCOUNT_D
(
  ROW_WID,
  GL_ACCOUNT_CAT_CODE,
  FIN_STMT_ITEM_CODE,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  PROFIT_CENTER_NUM,
  PROFIT_CENTER_ATTRIB,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  MANAGEMENT_SEGMENT_NUM,
  COMPANY_CODE,
  COMPANY_NAME,
  GROUP_ACCOUNT_NUM,
  RECON_TYPE_CODE,
  TAX_CAT_CODE,
  TAX_CAT_NAME,
  CURRENCY_CODE,
  CURRENCY_NAME,
  ACTIVE_FLG,
  ACCOUNT_SEG1_CODE,
  ACCOUNT_SEG1_ATTRIB,
  ACCOUNT_SEG1_INTEGRATION_ID,
  ACCOUNT_SEG2_CODE,
  ACCOUNT_SEG2_ATTRIB,
  ACCOUNT_SEG2_INTEGRATION_ID,
  ACCOUNT_SEG3_CODE,
  ACCOUNT_SEG3_ATTRIB,
  ACCOUNT_SEG3_INTEGRATION_ID,
  ACCOUNT_SEG4_CODE,
  ACCOUNT_SEG4_ATTRIB,
  ACCOUNT_SEG4_INTEGRATION_ID,
  ACCOUNT_SEG5_CODE,
  ACCOUNT_SEG5_ATTRIB,
  ACCOUNT_SEG5_INTEGRATION_ID,
  ACCOUNT_SEG6_CODE,
  ACCOUNT_SEG6_ATTRIB,
  ACCOUNT_SEG6_INTEGRATION_ID,
  ACCOUNT_SEG7_CODE,
  ACCOUNT_SEG7_ATTRIB,
  ACCOUNT_SEG7_INTEGRATION_ID,
  ACCOUNT_SEG8_CODE,
  ACCOUNT_SEG8_ATTRIB,
  ACCOUNT_SEG8_INTEGRATION_ID,
  ACCOUNT_SEG9_CODE,
  ACCOUNT_SEG9_ATTRIB,
  ACCOUNT_SEG9_INTEGRATION_ID,
  ACCOUNT_SEG10_CODE,
  ACCOUNT_SEG10_ATTRIB,
  ACCOUNT_SEG10_INTEGRATION_ID,
  ACCOUNT_SEG11_CODE,
  ACCOUNT_SEG11_ATTRIB,
  ACCOUNT_SEG11_INTEGRATION_ID,
  ACCOUNT_SEG12_CODE,
  ACCOUNT_SEG12_ATTRIB,
  ACCOUNT_SEG12_INTEGRATION_ID,
  ACCOUNT_SEG13_CODE,
  ACCOUNT_SEG13_ATTRIB,
  ACCOUNT_SEG13_INTEGRATION_ID,
  ACCOUNT_SEG14_CODE,
  ACCOUNT_SEG14_ATTRIB,
  ACCOUNT_SEG14_INTEGRATION_ID,
  ACCOUNT_SEG15_CODE,
  ACCOUNT_SEG15_ATTRIB,
  ACCOUNT_SEG15_INTEGRATION_ID,
  ACCOUNT_SEG16_CODE,
  ACCOUNT_SEG16_ATTRIB,
  ACCOUNT_SEG16_INTEGRATION_ID,
  ACCOUNT_SEG17_CODE,
  ACCOUNT_SEG17_ATTRIB,
  ACCOUNT_SEG17_INTEGRATION_ID,
  ACCOUNT_SEG18_CODE,
  ACCOUNT_SEG18_ATTRIB,
  ACCOUNT_SEG18_INTEGRATION_ID,
  ACCOUNT_SEG19_CODE,
  ACCOUNT_SEG19_ATTRIB,
  ACCOUNT_SEG19_INTEGRATION_ID,
  ACCOUNT_SEG20_CODE,
  ACCOUNT_SEG20_ATTRIB,
  ACCOUNT_SEG20_INTEGRATION_ID,
  ACCOUNT_SEG21_CODE,
  ACCOUNT_SEG21_ATTRIB,
  ACCOUNT_SEG21_INTEGRATION_ID,
  ACCOUNT_SEG22_CODE,
  ACCOUNT_SEG22_ATTRIB,
  ACCOUNT_SEG22_INTEGRATION_ID,
  ACCOUNT_SEG23_CODE,
  ACCOUNT_SEG23_ATTRIB,
  ACCOUNT_SEG23_INTEGRATION_ID,
  ACCOUNT_SEG24_CODE,
  ACCOUNT_SEG24_ATTRIB,
  ACCOUNT_SEG24_INTEGRATION_ID,
  ACCOUNT_SEG25_CODE,
  ACCOUNT_SEG25_ATTRIB,
  ACCOUNT_SEG25_INTEGRATION_ID,
  ACCOUNT_SEG26_CODE,
  ACCOUNT_SEG26_ATTRIB,
  ACCOUNT_SEG26_INTEGRATION_ID,
  ACCOUNT_SEG27_CODE,
  ACCOUNT_SEG27_ATTRIB,
  ACCOUNT_SEG27_INTEGRATION_ID,
  ACCOUNT_SEG28_CODE,
  ACCOUNT_SEG28_ATTRIB,
  ACCOUNT_SEG28_INTEGRATION_ID,
  ACCOUNT_SEG29_CODE,
  ACCOUNT_SEG29_ATTRIB,
  ACCOUNT_SEG29_INTEGRATION_ID,
  ACCOUNT_SEG30_CODE,
  ACCOUNT_SEG30_ATTRIB,
  ACCOUNT_SEG30_INTEGRATION_ID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  SET_ID,
  TENANT_ID,
  X_CUSTOM,
  PROJECT_CODE,
  PROJECT_ATTRIB,
  PROJECT_INTEGRATION_ID,
  PROGRAM_CODE,
  PROGRAM_ATTRIB,
  PROGRAM_INTEGRATION_ID,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT,
  CURRENT_FLG,
  ETL_PROC_WID
)
SELECT
  row_number() OVER (ORDER BY INTEGRATION_ID) + 
  COALESCE((SELECT MAX(ROW_WID) FROM workspace.oracle_edw.W_GL_ACCOUNT_D), 0) AS ROW_WID,
  GL_ACCOUNT_CAT_CODE,
  FIN_STMT_ITEM_CODE,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  PROFIT_CENTER_NUM,
  PROFIT_CENTER_ATTRIB,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  MANAGEMENT_SEGMENT_NUM,
  COMPANY_CODE,
  COMPANY_NAME,
  GROUP_ACCOUNT_NUM,
  RECON_TYPE_CODE,
  TAX_CAT_CODE,
  TAX_CAT_NAME,
  CURRENCY_CODE,
  CURRENCY_NAME,
  ACTIVE_FLG,
  ACCOUNT_SEG1_CODE,
  ACCOUNT_SEG1_ATTRIB,
  ACCOUNT_SEG1_INTEGRATION_ID,
  ACCOUNT_SEG2_CODE,
  ACCOUNT_SEG2_ATTRIB,
  ACCOUNT_SEG2_INTEGRATION_ID,
  ACCOUNT_SEG3_CODE,
  ACCOUNT_SEG3_ATTRIB,
  ACCOUNT_SEG3_INTEGRATION_ID,
  ACCOUNT_SEG4_CODE,
  ACCOUNT_SEG4_ATTRIB,
  ACCOUNT_SEG4_INTEGRATION_ID,
  ACCOUNT_SEG5_CODE,
  ACCOUNT_SEG5_ATTRIB,
  ACCOUNT_SEG5_INTEGRATION_ID,
  ACCOUNT_SEG6_CODE,
  ACCOUNT_SEG6_ATTRIB,
  ACCOUNT_SEG6_INTEGRATION_ID,
  ACCOUNT_SEG7_CODE,
  ACCOUNT_SEG7_ATTRIB,
  ACCOUNT_SEG7_INTEGRATION_ID,
  ACCOUNT_SEG8_CODE,
  ACCOUNT_SEG8_ATTRIB,
  ACCOUNT_SEG8_INTEGRATION_ID,
  ACCOUNT_SEG9_CODE,
  ACCOUNT_SEG9_ATTRIB,
  ACCOUNT_SEG9_INTEGRATION_ID,
  ACCOUNT_SEG10_CODE,
  ACCOUNT_SEG10_ATTRIB,
  ACCOUNT_SEG10_INTEGRATION_ID,
  ACCOUNT_SEG11_CODE,
  ACCOUNT_SEG11_ATTRIB,
  ACCOUNT_SEG11_INTEGRATION_ID,
  ACCOUNT_SEG12_CODE,
  ACCOUNT_SEG12_ATTRIB,
  ACCOUNT_SEG12_INTEGRATION_ID,
  ACCOUNT_SEG13_CODE,
  ACCOUNT_SEG13_ATTRIB,
  ACCOUNT_SEG13_INTEGRATION_ID,
  ACCOUNT_SEG14_CODE,
  ACCOUNT_SEG14_ATTRIB,
  ACCOUNT_SEG14_INTEGRATION_ID,
  ACCOUNT_SEG15_CODE,
  ACCOUNT_SEG15_ATTRIB,
  ACCOUNT_SEG15_INTEGRATION_ID,
  ACCOUNT_SEG16_CODE,
  ACCOUNT_SEG16_ATTRIB,
  ACCOUNT_SEG16_INTEGRATION_ID,
  ACCOUNT_SEG17_CODE,
  ACCOUNT_SEG17_ATTRIB,
  ACCOUNT_SEG17_INTEGRATION_ID,
  ACCOUNT_SEG18_CODE,
  ACCOUNT_SEG18_ATTRIB,
  ACCOUNT_SEG18_INTEGRATION_ID,
  ACCOUNT_SEG19_CODE,
  ACCOUNT_SEG19_ATTRIB,
  ACCOUNT_SEG19_INTEGRATION_ID,
  ACCOUNT_SEG20_CODE,
  ACCOUNT_SEG20_ATTRIB,
  ACCOUNT_SEG20_INTEGRATION_ID,
  ACCOUNT_SEG21_CODE,
  ACCOUNT_SEG21_ATTRIB,
  ACCOUNT_SEG21_INTEGRATION_ID,
  ACCOUNT_SEG22_CODE,
  ACCOUNT_SEG22_ATTRIB,
  ACCOUNT_SEG22_INTEGRATION_ID,
  ACCOUNT_SEG23_CODE,
  ACCOUNT_SEG23_ATTRIB,
  ACCOUNT_SEG23_INTEGRATION_ID,
  ACCOUNT_SEG24_CODE,
  ACCOUNT_SEG24_ATTRIB,
  ACCOUNT_SEG24_INTEGRATION_ID,
  ACCOUNT_SEG25_CODE,
  ACCOUNT_SEG25_ATTRIB,
  ACCOUNT_SEG25_INTEGRATION_ID,
  ACCOUNT_SEG26_CODE,
  ACCOUNT_SEG26_ATTRIB,
  ACCOUNT_SEG26_INTEGRATION_ID,
  ACCOUNT_SEG27_CODE,
  ACCOUNT_SEG27_ATTRIB,
  ACCOUNT_SEG27_INTEGRATION_ID,
  ACCOUNT_SEG28_CODE,
  ACCOUNT_SEG28_ATTRIB,
  ACCOUNT_SEG28_INTEGRATION_ID,
  ACCOUNT_SEG29_CODE,
  ACCOUNT_SEG29_ATTRIB,
  ACCOUNT_SEG29_INTEGRATION_ID,
  ACCOUNT_SEG30_CODE,
  ACCOUNT_SEG30_ATTRIB,
  ACCOUNT_SEG30_INTEGRATION_ID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CAST(CREATED_ON_DT AS TIMESTAMP) AS CREATED_ON_DT,
  CAST(CHANGED_ON_DT AS TIMESTAMP) AS CHANGED_ON_DT,
  CAST(AUX1_CHANGED_ON_DT AS TIMESTAMP) AS AUX1_CHANGED_ON_DT,
  CAST(AUX2_CHANGED_ON_DT AS TIMESTAMP) AS AUX2_CHANGED_ON_DT,
  CAST(AUX3_CHANGED_ON_DT AS TIMESTAMP) AS AUX3_CHANGED_ON_DT,
  CAST(AUX4_CHANGED_ON_DT AS TIMESTAMP) AS AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  SET_ID,
  TENANT_ID,
  X_CUSTOM,
  PROJECT_CODE,
  PROJECT_ATTRIB,
  PROJECT_INTEGRATION_ID,
  PROGRAM_CODE,
  PROGRAM_ATTRIB,
  PROGRAM_INTEGRATION_ID,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT,
  CURRENT_FLG,
  2625278 AS ETL_PROC_WID
FROM I$_2625278_1
WHERE IND_UPDATE = 'I'

In [ ]:
%sql
-- Step 3: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SILOS_SIL_GLACCOUNTDIMENSION' AS PACKAGE_NAME,
    'W_GL_ACCOUNT_D' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2625278 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    '1' AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 4: Insert to W_ETL_LOAD_DATES_LOG
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2625278 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 5: Drop flow table
DROP VIEW IF EXISTS I$_2625278_1